<a href="https://colab.research.google.com/github/bdi2357/CoverLetter/blob/main/CoverLetrrer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai

In [2]:
!git clone https://github.com/bdi2357/CoverLetter

Cloning into 'CoverLetter'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 24 (delta 8), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (24/24), 84.11 KiB | 16.82 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [3]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [4]:
!pip install -q PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.9 MB/s eta 0:00:00


In [5]:
!pip install docx
!pip install fpdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53894 sha256=5c8ff149c93984de5796f761bf717d71a25006b5c3cd1bb8e2066779d5be489f
  Stored in directory: /root/.cache/pip/wheels/81/f5/1d/e09ba2c1907a43a4146d1189ae4733ca1a3bfe27ee39507767
Successfully built docx
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=807ddede081199ea7d02174eb5bff47fb34b9fc0e9de1da6f1261a45ac2e126a
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [6]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 11.7 MB/s eta 0:00:00


In [7]:
#from genai import GenerativeModel
from docx import Document
from docx.shared import Pt
import re
import time

In [8]:
# Used to securely store your API key
from google.colab import userdata

In [9]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [10]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [11]:
model = genai.GenerativeModel('gemini-1.5-flash')

In [12]:
from PyPDF2 import PdfReader

def load_and_extract_text(file_path):
    with open(file_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
        return text

In [13]:
def generate_cover_letter(model,cv_text, job_description_text):

    prompt = f"""Based on the provided job description and CV, write a compelling and personalized cover letter that highlights relevant skills and experiences.

    Job Description:\n{job_description_text}\n\nCV:\n{cv_text}\n\n

    Tailor the letter to the specific job requirements and showcase the candidate's enthusiasm for the position. Be sure to mention specific accomplishments and quantify results whenever possible. Keep the tone professional and engaging.
    As concise as possible no more than 4 sentences. Avoid using adjectives unless neccessary , as less as possible , do not use emotive adjectives in any case, use only factual information based on the cv_text"""
    print("HERE")
    response = model.generate_content(prompt)  # Remove the 'context' argument
    return response

In [14]:
job_description = """ We're seeking an AI Developer to join our team. In this role, you'll leverage artificial intelligence and machine learning techniques to improve the invoice reconciliation process and create a unified data format across various financial systems.

Responsibilities:
- Develop and implement AI and machine learning models to automate invoice reconciliation
- Create intelligent systems to unify diverse invoice data into a standardized format
- Design and build natural language processing (NLP) solutions to extract key information from unstructured invoice data
- Implement machine learning algorithms to identify patterns, anomalies, and potential errors in financial data
- Collaborate with finance and accounting teams to understand business requirements and integrate AI solutions into existing workflows
- Continuously improve and optimize AI models based on new data and changing business needs

Requirements:
- Masters or PhD in Computer Science, Artificial Intelligence, or related field
- 3+ years of experience developing AI and machine learning solutions, preferably in finance or accounting domains
- Strong programming skills in Python and experience with ML frameworks (e.g., TensorFlow, PyTorch, scikit-learn)
- Experience with NLP techniques and text analysis
- Familiarity with financial systems, ERP software, and accounting principles
- Knowledge of data privacy and security best practices
- Excellent problem-solving skills and ability to translate complex business requirements into technical solutions

No agencies please.
This individual must be available for 30h/week and during UK working hours.

"""

In [15]:
cv_text = load_and_extract_text("CoverLetter/Data/CV_GPT_rev.pdf")

In [16]:
generate_cover_letter(model,cv_text, job_description).text

HERE


"Dear Hiring Manager,\n\nI am writing to express my keen interest in the AI Developer position at your company. With over 6 years of experience developing and deploying machine learning solutions, particularly in finance and data analysis, I am confident in my ability to contribute significantly to your team. My expertise in NLP, financial data analysis, and experience in building predictive models from scratch, including deployment, aligns perfectly with the requirements of this role. I am eager to leverage my skills to automate invoice reconciliation, standardize data formats, and enhance your company's financial processes. \n"

In [17]:
def extract_key_phrases(model, cover_letter_text, job_description_text, num_phrases=4):
    """
    Extracts key phrases from a cover letter based on their relevance to a job description using Gemini.

    Args:
        model: The Gemini language model instance.
        cover_letter_text: The text of the cover letter.
        job_description_text: The text of the job description.
        num_phrases: The number of phrases to extract (default is 4).

    Returns:
        A list of the most important phrases.
    """

    prompt = f"""Given the following cover letter and job description, identify the {num_phrases} most important words or two-word expressions in the cover letter that are most relevant to the job description.
    These words or Two words expression should show professional skills of the candidate they should reflecy the candidate as a doer, if it is an action like developing/programming it should be accompanied with the subject of the action. such as :
    developing AI models.

    Cover Letter:
    {cover_letter_text}

    Job Description:
    {job_description_text}
    """
    print("HERE")
    response = model.generate_content(prompt)

    # Extract key phrases from response
    important_phrases = response.text.split('\n')  # Assuming each phrase is on a new line
    return important_phrases

In [18]:
time.sleep(3)
job_description = job_description #load_and_extract_text("job_description.pdf")
cv_text = load_and_extract_text("CoverLetter/Data/CV_GPT_rev.pdf")
cover_letter_text = generate_cover_letter(model,cv_text, job_description).text
for k in cover_letter_text.split("."):
  print(k)
time.sleep(3)
key_phrases = extract_key_phrases(model, cover_letter_text, job_description, num_phrases=3)
for k in key_phrases:
  print("#"*30)
  print(k)

HERE
Dear Hiring Manager,

My experience as a Full Stack Machine Learning Engineer, particularly in the financial domain, aligns strongly with the requirements for the AI Developer position
 I have extensive experience developing and deploying predictive models, integrating data from diverse sources, and creating unified formats
 My work with portfolio management and investment analysis tools at Finzor Ltd
 demonstrates my ability to translate complex business requirements into technical solutions
 

I am eager to leverage my expertise in AI and machine learning to contribute to your team's efforts in automating invoice reconciliation and standardizing financial data
 

HERE
##############################
Here are the 3 most important words/phrases from the cover letter, aligned with the job description, and highlighting the candidate's skills:
##############################

##############################
1. **Developing predictive models:**  This directly aligns with the job descript

In [19]:
def extract_strings(key_phrases):
  R= []
  for k in key_phrases:
    A = (re.findall('\*\*.*?\*\*',k))
    if len(A)>0:
      R += re.findall('[A-Za-z\ 0-9]+',A[0])
  return R

In [20]:
extract_strings(extract_key_phrases(model, cover_letter_text, job_description, num_phrases=3))

HERE


['Developing and deploying predictive models',
 'Integrating data from diverse sources',
 'Translating complex business requirements into technical solutions']

In [21]:
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from docx.oxml import OxmlElement

def bold_strings_in_text(text, bold_strings, output_doc):
    # Create a new Document
    doc = Document()

    # Add the text to the document, with specified strings in bold
    para = doc.add_paragraph()
    parts = split_text(text, bold_strings)

    for part in parts:
        if part.lower() in [s.lower() for s in bold_strings]:
            run = para.add_run(part)
            run.bold = True
        else:
            para.add_run(part)

    # Save the document
    doc.save(output_doc)

def split_text(text, bold_strings):
    import re
    pattern = '|'.join(re.escape(s) for s in bold_strings)
    parts = re.split(f'({pattern})', text)
    print(parts)
    return parts

# Example usage
text = "This is a sample text. We want some words like sample and words to be bold."
bold_strings = ["sample", "words"]
bold_strings_in_text(text, bold_strings,"output.docx")



['This is a ', 'sample', ' text. We want some ', 'words', ' like ', 'sample', ' and ', 'words', ' to be bold.']


In [22]:
cl_text = generate_cover_letter(model,cv_text, job_description).text
strings = extract_strings(extract_key_phrases(model, cl_text, job_description, num_phrases=3))
bold_strings_in_text(cl_text, strings, "test2.docx")

HERE
HERE
['Dear Hiring Manager,\n\nMy extensive experience in developing and deploying machine learning solutions, particularly in finance and data engineering, makes me an ideal candidate for the AI Developer position.  I possess a strong foundation in Python, ML frameworks (TensorFlow, PyTorch, scikit-learn), and ', 'NLP techniques', ", demonstrated through projects like Finzor and Palo Alto Networks. I'm eager to contribute my expertise to automating invoice reconciliation and creating a unified data format, leveraging my experience with data integration and model development. \n"]


In [23]:
print(strings)

['Developing AI models', 'NLP techniques', 'Data integration']


In [24]:
cl_text

"Dear Hiring Manager,\n\nMy extensive experience in developing and deploying machine learning solutions, particularly in finance and data engineering, makes me an ideal candidate for the AI Developer position.  I possess a strong foundation in Python, ML frameworks (TensorFlow, PyTorch, scikit-learn), and NLP techniques, demonstrated through projects like Finzor and Palo Alto Networks. I'm eager to contribute my expertise to automating invoice reconciliation and creating a unified data format, leveraging my experience with data integration and model development. \n"

In [25]:
!ls

CoverLetter  output.docx  sample_data  test2.docx


In [26]:
from docx import Document
import re

def bold_strings_in_text(text, bold_strings, output_doc):
    # Create a new Document
    doc = Document()

    # Add the text to the document, with specified strings in bold
    para = doc.add_paragraph()
    parts = split_text(text, bold_strings)

    for part in parts:
        run = para.add_run(part)
        if part.lower() in [s.lower() for s in bold_strings]:
            run.bold = True

    # Save the document
    doc.save(output_doc)

def split_text(text, bold_strings):
    pattern = '|'.join(re.escape(s) for s in bold_strings)
    parts = re.split(f'({pattern})', text, flags=re.IGNORECASE)
    return parts

# Example usage
text = """Dear Hiring Manager,

I am writing to express my strong interest in the AI Developer position at your company. My extensive experience in developing and deploying AI solutions, particularly in finance and data analytics, aligns well with your requirements. My accomplishments include building predictive models from scratch, integrating data from diverse sources into unified formats, and creating adaptive models for updating anti-malware filters. I am confident that my skills and experience would be a valuable asset to your team.
"""
bold_strings = ["Developing and deploying predictive models", "Integrating diverse data sources", "Creating intelligent systems for financial analysis"]
bold_strings_in_text(text, bold_strings, "output.docx")


In [27]:
from docx import Document
import re

def bold_strings_in_text(text, bold_strings, output_doc):
    # Create a new Document
    doc = Document()

    # Add the text to the document, with specified strings in bold
    para = doc.add_paragraph()
    parts = split_text(text, bold_strings)

    for part in parts:
        run = para.add_run(part)
        # Check if the part matches any bold string, ignoring case
        if any(re.fullmatch(re.escape(b), part, re.IGNORECASE) for b in bold_strings):
            run.bold = True

    # Save the document
    doc.save(output_doc)

def split_text(text, bold_strings):
    # Create a regex pattern to match any of the bold strings, case-insensitive
    pattern = '|'.join(re.escape(s) for s in bold_strings)
    parts = re.split(f'({pattern})', text, flags=re.IGNORECASE)
    return parts

# Example usage
text = """Dear Hiring Manager,

I am writing to express my strong interest in the AI Developer position at your company. My extensive experience in developing and deploying AI solutions, particularly in finance and data analytics, aligns well with your requirements. My accomplishments include building predictive models from scratch, integrating data from diverse sources into unified formats, and creating adaptive models for updating anti-malware filters. I am confident that my skills and experience would be a valuable asset to your team.
"""
bold_strings = ["developing and deploying AI solutions", "integrating data from diverse sources", "creating adaptive models"]
bold_strings_in_text(text, bold_strings, "output_bold.docx")
